In [3]:
# problem 3 a
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize

# this is Rosenbrock function
def rosenbrock(x):
    return 100.0 * (x[1] - x[0]**2.0)**2.0 + (1 - x[0])**2.0

# this is the gradient of Rosenbrock function
def rosenbrock_grad(x):
    return np.array([-400.0*x[0]*(x[1]-x[0]**2.0) - 2.0*(1-x[0]), 200.0*(x[1]-x[0]**2.0)])

# this is the hessian matrix of rosenbrock function
def rosenbrock_hess(x):
    return np.array([[1200.0*x[0]**2.0 - 400.0*x[1] + 2, -400.0*x[0]], [-400.0*x[0], 200.0]])

# this is Newton method for optimization
def newtons_method(start_point, tol=1e-5, max_iter=1000):
    x = start_point
    for i in range(max_iter):
        grad = rosenbrock_grad(x)
        hess = rosenbrock_hess(x)
        if np.linalg.norm(grad) < tol:
            break
        delta = np.linalg.solve(hess, -grad)
        x = x + delta
    return x, i

# test with different starting points
starting_points = [(-1, -1), (0, 0), (2, 2), (1.5, 1.5)]


results = []
for point in starting_points:
    result, iterations = newtons_method(np.array(point))
    results.append((result, iterations))

for i, (start_point, result) in enumerate(zip(starting_points, results)):
    solution, iterations = result
    print(f'Starting point: {start_point}, Solution: {solution}, Iterations: {iterations}')


Starting point: (-1, -1), Solution: [1. 1.], Iterations: 5
Starting point: (0, 0), Solution: [1. 1.], Iterations: 2
Starting point: (2, 2), Solution: [1. 1.], Iterations: 5
Starting point: (1.5, 1.5), Solution: [1. 1.], Iterations: 5


In [4]:
# Pr3 b

def steepest_descent(start_point, tol=1e-5, max_iter=1000, alpha=1e-3):
    x = np.array(start_point, dtype='float64')
    for i in range(max_iter):
        grad = rosenbrock_grad(x)
        if np.linalg.norm(grad) < tol:
            break
        x -= alpha * grad
    return x, i

# run the steepest descent optimization with the same starting points
results_sd = []
for point in starting_points:
    result_sd, iterations_sd = steepest_descent(point)
    results_sd.append((result_sd, iterations_sd))

# display the results for steepest descent
results_sd_formatted = []
for i, (start_point, result_sd) in enumerate(zip(starting_points, results_sd)):
    solution_sd, iterations_sd = result_sd
    results_sd_formatted.append(f'Starting point: {start_point}, Solution: {solution_sd}, Iterations: {iterations_sd}')

results_sd_formatted


['Starting point: (-1, -1), Solution: [0.66670739 0.4428965 ], Iterations: 999',
 'Starting point: (0, 0), Solution: [0.67388605 0.45255952], Iterations: 999',
 'Starting point: (2, 2), Solution: [1.20924655 1.46301622], Iterations: 999',
 'Starting point: (1.5, 1.5), Solution: [1.18405047 1.40263525], Iterations: 999']

In [8]:
# Implement the conjugate gradient algorithm using the Hestenes-Stiefel formula for beta
def conjugate_gradient_hs(f, grad_f, x0, tol=1e-5, max_iter=1000):
    x = np.array(x0, dtype='float64')
    r = -grad_f(x)
    d = r.copy()
    for i in range(max_iter):
        # Approximate the Hessian-vector product
        Hd = grad_f(x + tol * d) - grad_f(x) / tol
        alpha = np.dot(r, r) / np.dot(d, Hd) if np.dot(d, Hd) != 0 else 0
        x_new = x + alpha * d
        r_new = -grad_f(x_new)
        beta = np.dot(r_new, r_new - r) / np.dot(d, r_new - r) if np.dot(d, r_new - r) != 0 else 0
        d = r_new + beta * d
        x = x_new
        r = r_new
        if np.linalg.norm(r) < tol:
            break
    return x, i

# New starting points closer to the global minimum
new_starting_points = [(0.5, 0.5), (1.2, 1.2), (1.0, 1.0)]

# Test the conjugate gradient algorithm with Hestenes-Stiefel beta update and new starting points
new_results_cg_hs = []
for point in new_starting_points:
    result_cg_hs, iterations_cg_hs = conjugate_gradient_hs(
        rosenbrock, rosenbrock_grad, np.array(point))
    new_results_cg_hs.append((result_cg_hs, iterations_cg_hs))

# Format the new results for display
new_results_cg_hs_formatted = []
for i, (start_point, result_cg_hs) in enumerate(zip(new_starting_points, new_results_cg_hs)):
    solution_cg_hs, iterations_cg_hs = result_cg_hs
    new_results_cg_hs_formatted.append(f'Starting point: {start_point}, Solution: {solution_cg_hs}, Iterations: {iterations_cg_hs}')

new_results_cg_hs_formatted


['Starting point: (0.5, 0.5), Solution: [0.6271381  0.39334365], Iterations: 999',
 'Starting point: (1.2, 1.2), Solution: [1.11201358 1.23699203], Iterations: 999',
 'Starting point: (1.0, 1.0), Solution: [1. 1.], Iterations: 0']

The results suggest that Newton's Method is far more efficient for this problem, converging to the exact solution in very few iterations. This is  because Newton's Method uses second-order derivative information, which allows it to adjust its path towards the minimum more accurately and take larger steps when it is far from the minimum.

Both the Steepest Descent Method and the Conjugate Gradient Method did not converge within the given iteration limit, indicating they are less efficient for this particular function. Steepest Descent  perform poorly on functions with narrow valleys because it does not account for the curvature of the function, leading to a zigzagging path towards the minimum. The Conjugate Gradient Method is usually more efficient than Steepest Descent but still fell short for this function within the given number of iterations.

In [11]:
def run_gradient_descent(X, Y, learning_rate, iterations):
    theta = np.zeros(X.shape[1])
    for iteration in range(iterations):
        gradients = 2 / m * X.T.dot(X.dot(theta) - Y)
        theta -= learning_rate * gradients
    return theta

def run_stochastic_gradient_descent(X, Y, learning_rate, iterations):
    theta = np.zeros(X.shape[1])
    for iteration in range(iterations):
        for i in range(m):
            random_index = np.random.randint(m)
            xi = X[random_index:random_index+1]
            yi = Y[random_index:random_index+1]
            gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
            theta -= learning_rate * gradients
    return theta

# Step sizes to test
step_sizes = [0.5, 0.1, 0.03, 0.01, 0.003, 0.001]

# Comparing Gradient Descent and Stochastic Gradient Descent
for step_size in step_sizes:
    theta_gd = run_gradient_descent(X_b, Y, learning_rate=step_size, iterations=100)  # or your chosen number of iterations
    theta_sgd = run_stochastic_gradient_descent(X_b, Y, learning_rate=step_size, iterations=1)  # m iterations, one epoch
    print(f"Step size: {step_size}\nGradient Descent Theta: {theta_gd}\nStochastic Gradient Descent Theta: {theta_sgd}\n")


Step size: 0.5
Gradient Descent Theta: [ 0.995543   -1.97238924  2.99149022 -4.0019434 ]
Stochastic Gradient Descent Theta: [-2.54441846e+174  1.49035773e+174 -1.88336038e+174  3.01320103e+173]

Step size: 0.1
Gradient Descent Theta: [ 0.49458274 -1.46789209  2.56540647 -3.12589939]
Stochastic Gradient Descent Theta: [ 1.37799297 -2.11617436  2.99834238 -3.51941953]

Step size: 0.03
Gradient Descent Theta: [ 0.01266839 -0.75173761  1.19884794 -1.5580648 ]
Stochastic Gradient Descent Theta: [ 0.83271907 -1.78555078  2.9338468  -3.72548004]

Step size: 0.01
Gradient Descent Theta: [-0.18570309 -0.38330675  0.37640081 -0.69803057]
Stochastic Gradient Descent Theta: [ 0.86844614 -1.74383211  3.01248334 -3.95682521]

Step size: 0.003
Gradient Descent Theta: [-0.1694453  -0.17857416  0.0626198  -0.27857616]
Stochastic Gradient Descent Theta: [ 0.97795268 -2.00748292  3.03048364 -4.01002731]

Step size: 0.001
Gradient Descent Theta: [-0.08169768 -0.07314157  0.00858814 -0.10703609]
Stochastic

In comparing Gradient Descent and Stochastic Gradient Descent with  different step sizes showed GD converges more smoothly as it uses the whole dataset to calculate gradients, while SGD's performance is heavily dependent on step size, with large step sizes leading to divergence, as seen with 0.5. With smaller step sizes, SGD rapidly approached the true parameter values, even with an equivalent of a single GD iteration. This showcases SGD's efficiency for large datasets and its sensitivity to step size selection, requiring careful tuning to prevent divergence and achieve fast convergence.